<a href="https://colab.research.google.com/github/bphqk3/CF/blob/main/CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This aims to cluster only active players in the 2023-2024 season. Players will be dropped entirely in previous seasons if they're not in the league anymore**

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Apr  5 01:34:00 2024

@author: Brooke
"""

import pandas as pd
import time
from io import StringIO
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression, LassoCV, Lasso, RidgeCV, Ridge, ElasticNetCV, ElasticNet, BayesianRidge, LogisticRegression, SGDRegressor, RidgeClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score
from sklearn.experimental import enable_hist_gradient_boosting  # Enable the module
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import pickle
from scipy.stats import pearsonr
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import plotly.graph_objects as go

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(



# make consolidated season stats list

*   structures are different in each. Manually drop irrelevant or duplicated columns before merging per the stat in interest (PTS). Add prefixes where necessary





In [15]:
a = pd.read_parquet('/home/S2122/player_stat_history_traditional2122.parquet')
a

SyntaxError: invalid syntax (<ipython-input-15-bce4f1c105f8>, line 1)

In [ ]:
a = a.drop(['Name', 'Unnamed: 0', 'Team','GP', 'W', 'L','FGM','3PM','FTM','FP',
            'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
            'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
            'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'PF RANK',
            'FP RANK', 'DD2 RANK', 'TD3 RANK', '+/- RANK','+/-', 'GP RANK', 'W RANK', 'L RANK',
            'MIN RANK', 'PTS RANK'],axis = 1)

b = pd.read_parquet(r'C:/Users/Brooke/Desktop/rbt/rbt_player_history/nbacom/season_history/S2122/player_stat_history_drives2122.parquet')
b

In [ ]:
b = b.drop(['Name', 'Team', 'GP', 'W', 'L', 'MIN','FGM', 'FGA',
       'FG%', 'FTM', 'FTA', 'FT%', 'PTS', 'PTS%', 'PASS', 'PASS%', 'AST',
       'AST%', 'TO', 'TOV%', 'PF', 'PF%'], axis = 1)
b.rename(columns={'DRIVES':'DrivesPG'}, inplace = True)

#got rid of "misc" stats (c)

#got rid of "passing" stats (d)

e = pd.read_parquet(r'C:/Users/Brooke/Desktop/rbt/rbt_player_history/nbacom/season_history/S2122/player_stat_history_rebounding2122.parquet')
e

In [ ]:
e = e.drop(columns=['Name', 'Team', 'GP', 'W', 'L', 'MIN', 'REB', 'Contested REB',
       'Contested REB%', 'REB Chances'], axis = 1)
e = e.iloc[:, :-3]
e.rename(columns={'REB Chance%' :'REB Chance% PG'}, inplace = True)

f = pd.read_parquet(r'C:/Users/Brooke/Desktop/rbt/rbt_player_history/nbacom/season_history/S2122/player_stat_history_scoring2122.parquet')
f

In [ ]:
f = f.drop(columns=['Name', 'Unnamed: 0', 'Team', 'AGE', 'GP', 'W', 'L', 'MIN',
       '%FGA 2PT', '%FGA 3PT',
       '%PTS FBPs', '%PTS OFFTO', '%PTS PITP', '2FGM %AST',
       '2FGM %UAST', '3FGM %AST', '3FGM %UAST', 'FGM %AST', 'FGM %UAST',
       'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', '%FGA 2PT RANK',
       '%FGA 3PT RANK', '%PTS 2PT RANK','%PTS 3PT RANK',
       '%PTS FBPs RANK', '%PTS FT RANK', '%PTS OffTO RANK', '%PTS PITP RANK',
       '2FGM %AST RANK', '2FGM %UAST RANK', '3FGM %AST RANK',
       '3FGM %UAST RANK', 'FGM %AST RANK', 'FGM %UAST RANK'],axis = 1)
idrop = [2,5]
f = f.drop(f.columns[idrop], axis=1)
f.rename(columns={'%PTS 2PT':'%PTS 2PT PG', '%PTS 3PT':'%PTS 3PT PG','%PTS FT':'%PTS FT PG'}, inplace = True)

g = pd.read_parquet(r'C:/Users/Brooke/Desktop/rbt/rbt_player_history/nbacom/season_history/S2122/player_stat_history_shooting2122.parquet')
g


In [ ]:
g = g.drop(columns=['Name', 'Team', 'Age', 'RA_FGM','PiPnRA_FGM','MR_FGM','LC3_FGM','RC3_FGM','C3_FGM','AB3_FGM'],axis=1)
suffix = ' PG'
g.columns.values[1:] = [col + suffix for col in g.columns[1:]]
g = g.replace('-', 0)
g.iloc[:, 1:] = g.iloc[:, 1:].apply(pd.to_numeric)



#we can drop (i)
j = pd.read_parquet(r'C:/Users/Brooke/Desktop/rbt/rbt_player_history/nbacom/season_history/S2122/player_stat_history_advanced2122.parquet')
j

In [ ]:
j = j.drop(['Name', 'Unnamed: 0', 'Team', 'AGE', 'GP', 'W', 'L', 'MIN', 'AST/TO','POSS', 'FGM',
        'FGA', 'FG%', 'GP RANK', 'W RANK','L RANK', 'MIN RANK', 'OFFRTG RANK', 'DEFRTG RANK', 'NETRTG RANK',
        'AST% RANK', 'AST/TO RANK', 'OREB% RANK',
        'DREB% RANK', 'REB% RANK', 'eFG% RANK', 'TS% RANK',
        'USG% RANK', 'PACE RANK', 'PIE RANK', 'FGM RANK',
        'FG% RANK','OREB%','DREB%',],axis=1)
j = j.drop(j.columns[7], axis=1).iloc[:,:12]

h = pd.read_parquet(r'C:/Users/Brooke/Desktop/rbt/rbt_player_history/nbacom/season_history/S2122/player_stat_history_clutch-usage2122.parquet')
h

In [ ]:
h = h.drop(['Name', 'Unnamed: 0', 'Team', 'AGE', 'GP', 'W', 'L', '%FGM', '%FGA', '%3PM', '%3PA', '%FTM', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK',
          '%FTA', '%OREB','%DREB', '%TOV', '%STL', '%BLK', '%BLKA', '%PF', '%PFD','GP RANK', 'W RANK',
            'L RANK', 'MIN RANK', 'USG% RANK','%FGM RANK', '%FGA RANK', '%3PM RANK', '%3PA RANK', '%FTM RANK',
            '%FTA RANK', '%OREB RANK', '%DREB RANK', '%REB RANK', '%AST RANK','%TOV RANK',
            '%STL RANK', '%BLK RANK', '%BLKA RANK', '%PF RANK','%PFD RANK', '%PTS RANK','MIN'],axis = 1)

#capture the players that are clutch; fill non-clutch players with zeros for their stats when we merge
h['PLAYER'] = 1
new_names = [(i,'Clutch_Game_'+i) for i in h.iloc[:, 1:].columns.values]
h.rename(columns = dict(new_names), inplace=True)

k = pd.read_parquet(r'C:/Users/Brooke/Desktop/rbt/rbt_player_history/nbacom/season_history/S2122/player_stat_history_clutch-advanced2122.parquet')
k

In [ ]:
k = k.drop(['Name', 'Unnamed: 0', 'Team', 'AGE', 'GP', 'W', 'L', 'MIN', 'AST/TO', 'FGM',
            'OffRtg', 'DefRtg', 'NetRtg', 'AST%', 'REB%','eFG%',
            'FGA', 'FG%', 'GP RANK', 'W RANK','L RANK', 'MIN RANK',
            'AST% RANK', 'AST/TO RANK', 'OREB% RANK',
            'DREB% RANK', 'REB% RANK', 'eFG% RANK', 'TS% RANK',
            'USG% RANK', 'PACE RANK', 'PIE RANK', 'FGM RANK',
            'FG% RANK','OREB%','DREB%',],axis=1)
kdrop = [1,2]
k = k.drop(k.columns[kdrop], axis=1).iloc[:,:5]
new_names = [(i,'Clutch_Game_'+i) for i in k.iloc[:, 1:].columns.values]
k.rename(columns = dict(new_names), inplace=True)

l = pd.read_parquet(r'C:/Users/Brooke/Desktop/rbt/rbt_player_history/nbacom/season_history/S2122/player_stat_history_clutch-traditional2122.parquet')
l

In [ ]:
l = l.drop(['Name', 'Unnamed: 0', 'Team','GP', 'W', 'L','FGM','3PM','FTM','FP',
            'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
            'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
            'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'PF RANK',
            'DD2 RANK', 'TD3 RANK', '+/- RANK','+/-', 'GP RANK', 'W RANK', 'L RANK',
            'MIN RANK', 'PTS RANK','Age'],axis = 1)
new_names = [(i,'Clutch_Game_'+i) for i in l.iloc[:, 1:].columns.values]
l.rename(columns = dict(new_names), inplace=True)

S2122_Player_Stats = pd.merge(a, j, on='Player', how='outer').merge(e, on='Player', how='outer').merge(f, on='Player', how='outer').merge(g, on='Player', how='outer').merge(b, on='Player', how='outer').merge(h, on='Player', how='outer').merge(k, on='Player', how='outer').merge(l, on='Player', how='outer').fillna(0)

S2122_Player_Stats_noClutch = pd.merge(a, j, on='Player', how='outer').merge(e, on='Player', how='outer').merge(f, on='Player', how='outer').merge(g, on='Player', how='outer').merge(b, on='Player', how='outer').fillna(0)


#Player Names that meet the criteria to be analyzed (4/5/2024)


*   Filter players 2023-2024 season by avg min > 15 and number games played > 20
*   copy stats to "data" df with the 'Player' column removed so only numeric values exist





In [ ]:
S2324 = pd.read_parquet(r'C:\Users\Brooke\Desktop\rbt\rbt_player_history\player_full_stat_history.parquet')
S2324 = S2324.groupby('Player').filter(lambda x: x['MIN'].mean() > 15 and len(x) >= 20)
S2324_Player_Names = S2324['Player'].unique().tolist()

Players_2122 = S2122_Player_Stats_noClutch[S2122_Player_Stats_noClutch['Player'].isin(S2324_Player_Names)]
Players_2122 = Players_2122.copy()
Players_2122.sort_values(by=['Player'], inplace = True)
Players_2122.reset_index(drop=True,inplace = True)

data = Players_2122.copy()
data = data.drop(['Player'],axis=1)
data_col_names = data.columns

# Feature Correlation

In [ ]:
# Calculate the correlation matrix
cor_matrix = data.corr()

# Plot correlation heatmap using seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(cor_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation of NBA Stats')
plt.show()


# Filter for positive correlations and exclude diagonal
positive_corr = cor_matrix.mask(np.tril(np.ones(cor_matrix.shape, dtype=bool))).stack().sort_values(ascending=False)

# Filter for significant correlations (greater than 0.9)
significant_corr = positive_corr[positive_corr > 0.9]

# Get the first 10 pairs of positively correlated variables
top_corr = significant_corr
# Plot the cross-correlation
plt.figure(figsize=(10, 6))
top_corr.plot(kind='bar', color='skyblue')
plt.title('Top 10 Positively Correlated Variable Pairs')
plt.xlabel('Variable Pair')
plt.ylabel('Correlation Coefficient')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


significant_corr_pairs = []
for var1 in data.columns:
    for var2 in data.columns:
        if var1 < var2:
            corr, p_value = pearsonr(data[var1], data[var2])
            if np.abs(corr) > 0.5 and p_value < 0.05:  # Set the significance level at 5%
                significant_corr_pairs.append((var1, var2))

print(significant_corr_pairs[:10])
plt.show()

This code is performing cross-correlation analysis on pairs of variables in a dataset to identify significant correlations based on the Pearson
correlation coefficient and p-value. Here's a breakdown of what the code is doing:

Iterating Over Columns: The code iterates over each
column (variable) in the dataset twice using nested loops. This way, it considers all possible pairs of variables.

Calculating Correlation: For each pair of variables (var1, var2), it calculates the Pearson correlation coefficient (corr) and the p-value (p_value) using the pearsonr function from the scipy.stats module.

Checking Significance: It checks if the absolute value of the correlation coefficient is greater than 0.5 and if the p-value is less than 0.05 (significance level set at 5%). If both conditions are met, the pair (var1, var2) is considered a significant correlation pair.

Storing Significant Pairs: The significant correlation pairs are stored in the significant_corr_pairs list.

Displaying Results: The first 10 significant correlation pairs are printed, and then plt.show() is called to display any plots (though there are no plots generated in the provided code).

A significant correlation means that there is a statistically meaningful relationship between two variables. In the context of the provided code, significance is determined based on two criteria:

Correlation Coefficient: The absolute value of the Pearson correlation coefficient (corr) must be greater than 0.5. This indicates a strong linear relationship between the variables.

P-value: The p-value (p_value) must be less than 0.05, which is a common significance level used in statistical hypothesis testing. A p-value below this threshold suggests that the observed correlation coefficient is unlikely to have occurred by random chance alone.

Keeping both pairs of variables in your dataset depends on the specific context of your analysis and your goals. Here are some considerations:

Relevance: Are the variables meaningful and relevant to your analysis? If the variables are not relevant to your research question or analysis objectives, it may be unnecessary to keep them in the dataset.

Collinearity: If two variables are highly correlated, they may provide redundant information. In some cases, multicollinearity (high correlation between predictors) can cause issues in statistical modeling, such as inflated standard errors or unstable coefficient estimates. In such cases, you may choose to keep only one of the correlated variables or consider techniques like principal component analysis (PCA) to reduce dimensionality.

Interpretability: Consider whether the correlation between the variables makes sense in the context of your data and domain knowledge. Sometimes, statistically significant correlations may not have a clear causal explanation or practical significance.

Data Quality: Ensure that the observed correlations are not artifacts of data quality issues, such as outliers or measurement errors. It's important to carefully examine the data and consider potential confounding variables.

Ultimately, the decision to keep or remove correlated pairs of variables should be guided by your research objectives, domain knowledge, and the specific requirements of your analysis. If in doubt, it's often helpful to consult with domain experts or statisticians for guidance.

##Good news! I tried signifance levels of 1%, 10%, and 30% and calculated the same number of signficant corr pairs! Per chatgpt: Overall,observing the same number of significant correlations across different significance levels suggests that the relationships between variables are likely to be meaningful and robust. This suggests that the observed correlations are likely to be genuine and not simply due to random chance.However, it's important to interpret the results in conjunction with other factors and consider the implications for the research question and subsequent analyses.


# Boxplot Analysis

In [ ]:
num_cols = data.shape[1]
num_rows = 10
num_cols_per_row = 5

# Create a figure and axes
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols_per_row, figsize=(20, 30))

# Flatten the axes array to simplify iteration
axes = axes.flatten()

# Iterate over columns and create boxplots
for i, col in enumerate(data.columns):
    ax = axes[i]
    ax.boxplot(data[col])
    ax.set_title(col)
    ax.grid(True)

# Hide any empty subplots
for i in range(num_cols, num_rows * num_cols_per_row):
    axes[i].axis('off')

plt.tight_layout()
plt.show()

scaler = MinMaxScaler()
data = scaler.fit_transform(data)

## after this scaling, the df becomes an array! ##

axes = axes.flatten()

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols_per_row, figsize=(20, 4 * num_rows))

# Flatten the axes array to simplify iteration
axes = axes.flatten()

# Iterate over columns and create boxplots
for i in range(num_cols):
    ax = axes[i]
    ax.boxplot(data[:, i])
    ax.set_title(f'Column {i+1}')
    ax.grid(True)

# Hide any empty subplots
for i in range(num_cols, num_rows * num_cols_per_row):
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Perform Principal Component Analysis (PCA)

In [ ]:
pca = PCA().fit(data)
#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Explained Variance')
plt.show()
#n = 24 looks like it capture ~95+% variance
#n = 15 captured 90% of the variance from Osken/Onay paper with similar stats...

pca = PCA(n_components=24)
data_pca = pca.fit_transform(data)
with open('data_pca_n24.pkl', 'wb') as f:
    pickle.dump(data_pca, f)

#'pca' is dtype = decomposition
#'data_pca' is array with dtype float 64
components = pca.components_

num_pcs = 24
num_vars = components.shape[1]
num_rows = 6

plt.figure(figsize=(24,24))  # Adjust the figure size as needed

for i in range(num_pcs):
    plt.subplot(num_rows, num_pcs // num_rows, i+1)  # Create subplots in 6 rows
    plt.bar(range(num_vars), components[i])
    plt.xticks(range(num_vars), data_col_names,rotation=90)  # Rotate x-axis labels for better readability
    plt.title(f'Contribution of variables to PC{i+1}')

plt.suptitle('Contribution of NBA stats variables to the principal components')
plt.tight_layout()
plt.show()

# Implement k-means clustering algorithm with PCA data



We run our k-means algorithm for all values of k between 5 and 35.
To account for the potential impact of random centroid seeds on the cluster
robustness, we run each clustering instance of the algorithm 10 times, with
different random seeds, selecting the best performing clustering output,
using Python’s scikit learn library. For each of these clustering outputs,
we estimate the cluster validity using Silhouette value and Calinski-Harabasz
pseudo-F index for each run of the algorithm (Chan et al., 2012). Using these
two validity indices, we identify best performing k values as k = 25 and k = 26.
We also recognise that Bezdek and Bezdek (1981) shows that no cluster validity
measure is able to capture the correct number of underlying clusters
in every case. Therefore, we also randomly sample other k values to use as
inputs for our predictors. We find that as the k value deviates
from the 25–26 range in either direction, the prediction success
drops.

In [ ]:
# Define the range of k values
k_values = range(2, 40)

# Dictionary to store the evaluation metrics for each k
#The average silhouette score is commonly used to evaluate
#the overall quality of clustering across different numbers of clusters.
#can run other methods take a look at elbow method and intercluster distance maps by yellowbrick
silhouette_scores_data_pca = {}
avg_ss_data_pca = {}
calinski_scores_data_pca = {}

# Run k-means clustering 10 times for each k
for k in k_values:
    silhouette_scores_data_pca[k] = []
    avg_ss_data_pca[k]=[]
    calinski_scores_data_pca[k] = []
    for i in range(10):
        #Increasing n_init may provide a more stable and reliable solution
        #by exploring a larger range of initializations. It can help
        #mitigate the risk of getting stuck in a local minimum and improve the
        #chance of finding a solution closer to the global minimum of inertia.
        #n_init defaults to 10, higher numbers will cost a lot more...
        #50 is considered relatively large. try it and see what resources stand or not
        # Random state = None to ensure different seeds; centroids will be initialized
        #randomly each time you run the algorithm. (exploratory)
        # Random state = integer value to ensure different seeds; centroids will be initialized
        #randomly each time you run the algorithm. (need reproducability)
        kmeans = KMeans(n_clusters=k, n_init=10, random_state=None)
        labels = kmeans.fit_predict(data_pca)
        # Calculate silhouette score
        silhouette = silhouette_score(data_pca, labels)
        silhouette_scores_data_pca[k].append(silhouette)
        avg_ss = np.mean(silhouette)
        avg_ss_data_pca[k].append(avg_ss)
        # Calculate Calinski-Harabasz score
        calinski = calinski_harabasz_score(data_pca, labels)
        calinski_scores_data_pca[k].append(calinski)

# Select the best performing clustering output for each k
best_silhouette_data_pca = {k: max(silhouette_scores_data_pca[k]) for k in silhouette_scores_data_pca}
best_avg_ss_data_pca = {k: max(avg_ss_data_pca[k]) for k in avg_ss_data_pca[k]}
best_calinski_data_pca = {k: max(calinski_scores_data_pca[k]) for k in calinski_scores_data_pca}

# Print the best silhouette and Calinski-Harabasz scores for each k
print("Best Silhouette Scores:")
for k, score in best_silhouette_data_pca.items():
    print(f"k={k}: {score:.4f}")

print("Best Average Silhouette Scores:")
for k, score in best_avg_ss_data_pca.items():
    print(f"k={k}: {score:.4f}")

print("\nBest Calinski-Harabasz Scores:")
for k, score in best_calinski_data_pca.items():
    print(f"k={k}: {score:.4f}")


## At this point, you should choose how many clusters you want and you repeat with that n_clusters
#consider set random state?
k = 8
kmeans = KMeans(n_clusters=k, n_init=10, random_state=None)
kmeans.fit_predict(data_pca)
cluster_labels = kmeans.labels_
Players_2122['Cluster FUCKER'] = cluster_labels

# Define cluster names
cluster_names = {
    0: 'Rebounding Bigs',
    1: 'Midrange Scorers',
    2: 'Three Point Sharpshooters',
    3: 'Pass-First Defensive Guards',
    4: 'Low Usage Defensive Specialists',
    5: 'All-Around Forwards',
    6: 'Prehistoric Bigs',
    7: 'Ball Dominant Superstars'
}
Players_2122['Cluster'] = Players_2122['Cluster'].map(cluster_names)

# Plotting Clusters
plt.figure(figsize=(10, 8))
sns.scatterplot(x=data_pca[:, 0], y=data_pca[:, 1], hue=Players_2122['Cluster'])
plt.title('8 NBA Player Clusters')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.show()


km_centers = pd.DataFrame(kmeans.cluster_centers_, columns=data_pca.columns)  # SCALED cluster centers/means

# Name clusters before pivoting
km_centers['Cluster'] = [f'Cluster {i+1}' for i in range(k)]

# Massage data
km_centers = km_centers.rename(columns={'astPerMinute': 'AST', 'blkPerMinute': 'BLK',
                                        'drbPerMinute': 'DRB', 'fg2aPerMinute': '2PA',
                                        'fg3aPerMinute': '3PA', 'ftaPerMinute': 'FTA',
                                        'orbPerMinute': 'ORB', 'ptsPerMinute': 'PTS',
                                        'stlPerMinute': 'STL', 'tovPerMinute': 'TOV',
                                        'pctFT': 'FT%', 'pctFG2': '2P%', 'pctFG3': '3P%',
                                        'verticalLeapMaxInches': 'LEAP', 'repsBenchPress135': 'STR',
                                        'timeLaneAgility': 'AGL'})

km_centers_melted = km_centers.melt(id_vars='Cluster', var_name='feature', value_name='z_val')

# Reset the order of predictor variables for plotting
predictor_order = ['PTS', 'AST', 'ORB', 'DRB', 'STL', 'BLK', 'TOV', '2PA', '3PA', 'FTA',
                   '2P%', '3P%', 'FT%', 'LEAP', 'AGL', 'STR']
km_centers_melted['feature'] = pd.Categorical(km_centers_melted['feature'], categories=predictor_order)

# Reset the order of clusters for plotting
km_centers_melted['Cluster'] = pd.Categorical(km_centers_melted['Cluster'],
categories=[f'Cluster {i+1}' for i in range(k)])

import plotly.express as px

# Plot cluster 1 # attempting to make a plot like this:https://alexcstern.github.io/hoopDown.html
fig = px.scatter(km_centers_melted[km_centers_melted['Cluster'] == 'Cluster 1'],
                 x='feature', y='z_val', color='Cluster',
                 title='Visualizing K-Means Cluster Makeups - Cluster 1',
                 labels={'feature': 'Predictor', 'z_val': 'Cluster Center'},
                 color_discrete_map={'Cluster 1': '#232D4B'},
                 template='plotly_white')

fig.update_layout(xaxis_tickangle=45)

import seaborn as sns
import matplotlib.pyplot as plt

# Plot cluster centers for each predictor
sns.set(style="whitegrid")
g = sns.FacetGrid(km_centers, col="Cluster", col_wrap=3, height=4)
g.map(sns.scatterplot, "feature", "z_val", color='blue')

# Adjust plot aesthetics
g.set_axis_labels("Predictor", "Cluster Center")
g.set_titles(col_template="{col_name}")
g.fig.suptitle("Visualizing K-Means Cluster Makeups", y=1.05)
plt.xticks(rotation=90, fontsize=8)
plt.tight_layout()

plt.show()



#  Hierarchical Clustering

In [ ]:
# Try clustering by single link, centroid, and ward (D2)
hcl_single = linkage(y=nba_dist, method='single')
hcl_centroid = linkage(y=nba_dist, method='centroid')
#this is squared Euclidean distances
hcl_ward = linkage(y=nba_dist, method='ward')

# This is squared Euclidean distances
squared_dissimilarities = pdist(data, metric='sqeuclidean')
# Convert the result to a square distance matrix
hcl_ward_d2 = linkage(squared_dissimilarities, method='ward')

plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
plt.plot(hcl_single)
plt.title('Single Linkage Dendrogram')
plt.xlabel('Distance')
plt.ylabel('Data Points')
plt.show()

# Plot dendrogram for centroid linkage method
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
plt.plot(hcl_centroid)
plt.title('Link with the Centroid')
plt.xlabel('Distance')
plt.ylabel('Data Points')
plt.show()

# Plot dendrogram for Ward
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
threshold = 40  # Adjust this threshold value as needed
dendrogram(hcl_ward, orientation='top', labels=None, color_threshold=threshold)
plt.title('Link with the Ward')
plt.xlabel('Distance')
plt.ylabel('Data Points')
plt.show()

# Plot dendrogram for Ward D2
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
threshold = 20000  # Adjust this threshold value as needed
dendrogram(hcl_ward_d2, orientation='top', labels=None, color_threshold=threshold)
plt.title('Link with the Ward Squared Dissimilarities')
plt.xlabel('Distance')
plt.ylabel('Data Points')
plt.show()
